### O objetivo desse projeto é utilizar Redes Neuras Convolucionais (CNN) para a classificação de objetos de uma base de imagens bastante conhecida "CIFAR-10", que possui mais de 60.000 dados de imagens 32x32 contendo as seguintes categorias de imagens: avião, automóvel, passáro, gato, veado, cachorro, sapo, cavalo, navio e caminhão.
### Ao longo do projeto, vamos utilizar diferentes CNN, como a VGG16, VGG19, ResNet50, aplicando diferentes ténicas para melhorar o modelo, como Data Augmentation, que é capaz de realizar pequenas alterações nas imagens a fim de obter mais variações para o treinamento, ténicas de TransferLearning, que é capaz de utilizar um modelo que já foi treinado com o objetivo de poupar recursos para treinar novos modelos e também aproveitar seu desempenho.
### Então nosso projeto consiste nas seguintes etapas:


### - Carregar dados
### - Definir modelo Keras
### - Compilar modelo Keras
### - Ajustar (fit) modelo Keras
### - Avaliar (evaluate) modelo Keras
### - Previsão

Realizando as importações necessárias para iniciar o projeto, e ocultando possíveis warnings que são relevantes para o projeto.

In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from keras.utils import to_categorical
import matplotlib.pyplot as plt

### Carregando os dados da base CIFAR-10 e separando as imagens em treino e teste para realizar o treinamento do modelo e a avaliação da performance.

In [ ]:
cifar10 = keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Visualizando as primeiras imagens do Dataset com Matplotlib bem como o shape das Imagens. É possível perceber que as imagens estão em baixa resolução e o shape contém 60 mil amostras 32x32 com 3 canais de cores, que é o RGB.

In [ ]:
class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

fig, axs = plt.subplots(3, 3, figsize=(12, 8))
for i, ax in enumerate(axs.flat):
    ax.imshow(X_test[i])
    ax.set_title(f"Class: {class_names[y_test[i][0]]}")
    ax.axis("off")


fig.subplots_adjust(hspace=0.6)
print()
print('-='*30, 'IMAGENS CIFAR-10', '-='*30)
print()
plt.show()
print()
print('-='*30, 'SHAPE', '-='*30)
print()
print('Amostras de Treino: ', X_train.shape)
print('Amostras de Teste: ', y_train.shape)

### Normalizando o conjunto de dados X

#### O conjunto de dados X_train e X_test não possui valores normalizados, variando de 0 a 255, para normalizar os dados vamos dividi-los por 255 para os valores ficarem entre 0 e 1.

In [ ]:
X_train = (X_train.astype('float32') / 255.)
X_test = (X_test.astype('float32') / 255.)

### Transformar o y_test, e y_train em to_categorical:
#### O y_test e o y_train estão representando o índice da imagem que se trata o conjunto de dados, para o modelo treinado não identificar no treinamento alguma relação dos índices, vamos aplicar o one_hot com a ideia de binarizar a classificação da imagem. Por exemplo, o conjunto X' é um gato, que é representado pelo índice 3, quando aplicamos o one_hot o gato será representado pelo seguinte array: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], ou seja, atribui o valor 1 no índice 3 que é classificado para o gato e 0 para os demais. Essa técnicas pode melhorar o desemepenho do nosso modelo para o modelo não identificar relações na nossa variável target que não existe.


In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

### Definindo a arquitetura do modelo com Keras
#### Primeiramente, vamos utilizar um modelo bem simples com 8 camadas e 204.602 parâmetros, aplicando técnicas de regularização, como o Dropout, que consiste em aplicar um "esquecimento" de neurônios com o objetivo da rede não depender e dar um peso alto para algum neurônio, também aplicar o MaxPooling para reduzir a dimensionalidade das feature map e coletando as características mais ativas.

#### Também, vamos inicializar os pesos dos neurônios aleatoriamente com o parâmetro "kernel_initializer", pois quando aplicar o Gradiente, técnica para a variação dos pesos para a redução da função custo, não possuir a mesma variancia para todos os parâmetros, caso contrário todos os pesos seriam alterados da mesma forma.

In [ ]:
model = keras.models.Sequential()
model.add(layers.InputLayer(input_shape=(32, 32, 3)))
model.add(layers.Conv2D(32, (3, 3), activation="relu", padding="valid"))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(80, kernel_initializer='glorot_uniform', activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, kernel_initializer='glorot_uniform', activation="softmax"))

model.summary()

#### Definir nosso otimizador do Gradiente, a métrica e a função custo e treinamento o modelo.
#### Aplicando um batch size de 200 amostras para calcular o custo e com 10 épocas para atualização dos pesos. O parâmetro validation_data com verbose=1, serve para o modelo testar em cada época a sua performance com os dados de teste e msotrar na tela.

In [ ]:
optimizer = keras.optimizers.Adam(beta_1=0.8)
loss = keras.losses.CategoricalCrossentropy()
metric = keras.metrics.CategoricalAccuracy()

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=[metric],
)

hist = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=256,
    validation_data=(X_test, y_test),
    verbose=1
)

#### Com esse modelo simples foi possível atingir uma acurácia de 68%.
#### Vamos visualizar de forma gŕafica o desempenho do treinamento. É possível visualizar que a acurácia de teste performou melhor que a do treino, o que é muito bom para o modelo.

In [ ]:
acc_train = hist.history['categorical_accuracy']
acc_test = hist.history['val_categorical_accuracy']

plt.plot(acc_train, '-g', label='Train Accuracy')
plt.plot(acc_test, '-b', label='Test Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

### Aumentando artificialmente o conjunto de dados CIFAR-10
#### Com a técnicas data augmentation, é possível fazer pequenas variações nas imagens de treinamento, com o objetivo do modelo perceber outros padrões que se comportam como uma nova imagem. Entre essas alterações podem incluir: Zoom, Deslocamento da imagem e Rotação, que serão aplicadas em cada época do treinamento. É importante inicializar novamente a arquitetura antes.

In [ ]:
model = keras.models.Sequential()
model.add(layers.InputLayer(input_shape=(32, 32, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='valid'))
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='valid'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='valid'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='valid'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='valid'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='valid'))
model.add(layers.Flatten())
model.add(layers.Dense(120, kernel_initializer='glorot_uniform', activation="relu"))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10, kernel_initializer='glorot_uniform', activation="relu"))

model.summary()

In [ ]:
# Data augmentation
aug_data = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
aug_train = aug_data.flow(X_train, y_train, batch_size=200)

# Define optimizer, loss, and metrics
optimizer = keras.optimizers.Adam(beta_1=0.8)  # Adjusted beta_1, default is 0.9
loss = keras.losses.CategoricalCrossentropy()
metrics = [keras.metrics.CategoricalAccuracy()]  # Use a list

# Compile model
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics,
)

# Number of steps per epoch
n_steps = X_train.shape[0] // 200

# Train the model
hist = model.fit(
    aug_train,
    steps_per_epoch=n_steps,
    epochs=70,
    validation_data=(X_test, y_test),  # No need for batch_size here
    verbose=1
)


### Esse modelo atingiu 83% de acurácia nos dados de teste

In [ ]:
acc_train = hist.history['accuracy']
acc_test = hist.history['val_accuracy']

import matplotlib.pyplot as plt

plt.plot(acc_train, '-g', label='Train Accuracy')
plt.plot(acc_test, '-b', label='Test Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

### Testando com outros modelos: VGG19, ResNet50.
#### Vamos aplicar diferentes técnicas para utilizar o VGG19 e o ResNet50, uma deles é o early stopping com o parâmetro from epoch, essa técnica visa parar o modelos, caso uma condição de performance seja atingida, como o custo não variar muito. E o from epoch é quando utilizamos o Transfer Learning, para não interromper o treinamento cedo demais, então o early_stopping só vai ser aplicada quando passar pela época definida.

### Ajuste dos parâmetros